In [2]:
%config InlineBackend.figure_format ='retina'
from matplotlib.dates import date2num, num2date
from matplotlib.colors import ListedColormap
from matplotlib import dates as mdates
from matplotlib.patches import Patch
from matplotlib import pyplot as plt
from matplotlib import ticker

import pandas as pd
import numpy as np
import os

import sys

sys.path.insert(0, '../')

from global_config import config

results_dir   = config.get_property('results_dir')
data_dir      = config.get_property('data_dir')

#%% Plot Tong's default setting
SMALL_SIZE  = 22
MEDIUM_SIZE = 22
BIGGER_SIZE = 22

plt.rc('font', size=SMALL_SIZE, family='sans-serif', serif='Arial') # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)    # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels"
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
plt.rc('text')


In [5]:
path_to_frcst_1 = os.path.join(results_dir, "forecast", "arima")
path_to_frcst_2 = os.path.join(results_dir, "forecast", "eakf_model1")
path_to_frcst_3 = os.path.join(results_dir, "forecast", "eakf_model2")
path_to_frcst_4 = os.path.join(results_dir, "forecast", "eakf_model3")

evals_df        = pd.read_csv(os.path.join(results_dir, "forecast", "evaluation", "scores.csv"), parse_dates=["frsct_date"])
usa_df          = pd.read_csv(os.path.join(data_dir, "processed_data_us.csv"), parse_dates=["date"])
dates_forecasts = usa_df.date[11:].values


In [6]:
def create_df_ensemble(weights_df, forecast_df_list, name_models):
    e_df = [forecast_df_list[idx]* weights_df.loc[name_models[idx]]["weigth"] for idx in range(len(forecast_df_list))]
    e_df = sum(e_df)
    return e_df


In [7]:
all_past_df               = pd.read_csv(os.path.join(results_dir, "forecast", "evaluation",  "scores_all_past.csv"), parse_dates=["frsct_date"])
all_past_df["frsct_date"] = all_past_df["forecast_date"]

eq_weights_df             = pd.read_csv( os.path.join(results_dir, "forecast", "evaluation", "scores_equal_weights.csv"), parse_dates=["frsct_date"])
k_past_df                 = pd.read_csv(os.path.join(results_dir, "forecast", "evaluation",  "scores_K_past.csv"), parse_dates=["frsct_date"])
k_past_df["frsct_date"]   = k_past_df["forecast_date"]

k_past                    = ['2_past_ens', '3_past_ens', '4_past_ens', '5_past_ens', '6_past_ens']
k_past_df                 = pd.concat([k_past_df, all_past_df])

In [8]:
scores_models_df = pd.read_csv(os.path.join(results_dir, "forecast", "evaluation", "scores.csv"), parse_dates=["frsct_date"])

model_stats_df = scores_models_df.copy()
ens_stats_df   = k_past_df.copy()

eq_weights_resume_df = eq_weights_df

ens_stats_df         = ens_stats_df[ens_stats_df.method.isin(["2_past_ens", "3_past_ens", "4_past_ens", "5_past_ens", "6_past_ens", "all_past_ens"])]
ens_stats_df = ens_stats_df[ens_stats_df.eval_horizon_ensemble=="6m"]
plot_bars_df = pd.concat([model_stats_df, ens_stats_df, eq_weights_resume_df])


In [9]:
years          = ['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']
pre_vac_years  = years[:6]
post_vac_years = years[6:]


In [10]:
name_cols    = {"arima": "Arima", "eakf_model1": "Model 1", "eakf_model2": "Model 2", "eakf_model3": "Model 3",
                "2_past_ens": "Ens. (2 Months)", "3_past_ens": "Ens. (3 Months)", "4_past_ens": "Ens. (4 Months)",
                "5_past_ens": "Ens. (5 Months)", "6_past_ens": "Ens. (6 Months)", "all_past_ens": "Ens. (All past)", "equal_weights_ensemble": "Ens. (Eq. Weights)"}

plot_bars_df["method_plot"] = plot_bars_df["method"].apply(lambda x: name_cols[x])
plot_bars_df["period"]      = "All"
plot_bars_df["frsct_date"]  = pd.to_datetime(plot_bars_df["frsct_date"])
plot_bars_df['year']        = plot_bars_df.frsct_date.dt.year
plot_bars_df['year']        = plot_bars_df.year.apply(lambda x: str(x))


In [11]:
eval_horizons = ['1m', '2m', '3m', '4m', '5m', '6m']
models_plt = ["arima", "eakf_model1", "eakf_model2", "eakf_model3"]


In [12]:
import scipy.stats as stats

eh = eval_horizons[0]

for eh in eval_horizons:
    plt_bar_df = plot_bars_df[plot_bars_df.method.isin(models_plt)]
    data_test  = plt_bar_df[plt_bar_df.eval_horizon==eh]

    wis_arima  = data_test[data_test.method=="arima"].wis.values
    wis_model1 = data_test[data_test.method=="eakf_model1"].wis.values
    wis_model2 = data_test[data_test.method=="eakf_model2"].wis.values
    wis_model3 = data_test[data_test.method=="eakf_model3"].wis.values


    #perform the Wilcoxon-Signed Rank Test
    pval_1 = stats.wilcoxon(wis_arima, wis_model1).pvalue
    pval_2 = stats.wilcoxon(wis_arima, wis_model2).pvalue
    pval_3 = stats.wilcoxon(wis_arima, wis_model3).pvalue

    print("Eval horizon {}. ARIMA vs Model 1 {}, ARIMA vs Model 2 {}, ARIMA vs Model 3 {}".format(eh, pval_1, pval_2, pval_3))

Eval horizon 1m. ARIMA vs Model 1 2.2161217314726123e-21, ARIMA vs Model 2 2.1111306794400893e-18, ARIMA vs Model 3 2.364525260265854e-23
Eval horizon 2m. ARIMA vs Model 1 1.043374410051002e-24, ARIMA vs Model 2 5.0600992951711526e-24, ARIMA vs Model 3 3.2065929660393643e-27
Eval horizon 3m. ARIMA vs Model 1 1.0092015819140604e-27, ARIMA vs Model 2 2.01600143645266e-27, ARIMA vs Model 3 5.8963792641706845e-30
Eval horizon 4m. ARIMA vs Model 1 1.4635346704871436e-28, ARIMA vs Model 2 2.3323514663969288e-29, ARIMA vs Model 3 3.439074413870708e-31
Eval horizon 5m. ARIMA vs Model 1 1.012661536425003e-28, ARIMA vs Model 2 8.420425006243619e-31, ARIMA vs Model 3 1.0438753733088183e-31
Eval horizon 6m. ARIMA vs Model 1 9.234358392916452e-29, ARIMA vs Model 2 1.2939796355610422e-30, ARIMA vs Model 3 2.0562500186705338e-31


In [13]:
bars_all_period_df = []

name_cols    = {"arima": "Arima", "eakf_model1": "Model 1", "eakf_model2": "Model 2", "eakf_model3": "Model 3",
                "2_past_ens": "Ens. (2 Months)", "3_past_ens": "Ens. (3 Months)", "4_past_ens": "Ens. (4 Months)",
                "5_past_ens": "Ens. (5 Months)", "6_past_ens": "Ens. (6 Months)", "all_past_ens": "Ens. (All past)", "equal_weights_ensemble": "Ens. (Eq. Weights)"}

plot_bars_df["method_plot"] = plot_bars_df["method"].apply(lambda x: name_cols[x])
plot_bars_df["period"]      = "All"
plot_bars_df["frsct_date"]  = pd.to_datetime(plot_bars_df["frsct_date"])
plot_bars_df['year']        = plot_bars_df.frsct_date.dt.year
plot_bars_df['year']        = plot_bars_df.year.apply(lambda x: str(x))

bars_all_period_df.append(plot_bars_df)

models_plt = ["eakf_model3", "2_past_ens", "all_past_ens", "equal_weights_ensemble"]

plt_bar_df = plot_bars_df[plot_bars_df.method.isin(models_plt)][["wis", "eval_horizon", "frsct_date", "method", "period"]]

eh        = eval_horizons[0]

models = [("eakf_model3", "2_past_ens"), ("eakf_model3", "all_past_ens"), ("2_past_ens", "all_past_ens"),
            ("eakf_model3", "equal_weights_ensemble"), ("2_past_ens", "equal_weights_ensemble"), ("all_past_ens", "equal_weights_ensemble")]

pvals_df = pd.DataFrame(columns=["model1", "model2", "pval", "eh"])
for eh in eval_horizons:
    data_test = plt_bar_df[plt_bar_df.eval_horizon==eh]
    for model1, model2 in models:
            dates1 = data_test[data_test.method==model1].frsct_date
            dates2 = data_test[data_test.method==model2].frsct_date
            dates  = list(set(dates1).intersection(dates2))

            data_use   = data_test[data_test.frsct_date.isin(dates)]
            wis_model1 = data_use[data_use.method==model1].wis.values
            wis_model2 = data_use[data_use.method==model2].wis.values

            pval       = stats.wilcoxon(wis_model1, wis_model2).pvalue

            pvals_df= pvals_df.append({"model1": model1, "model2": model2, "pval": pval, "eh": eh}, ignore_index=True)

pvals_df["column_latex"] = pvals_df.apply(lambda x: "{} vs {}".format(name_cols[x.model1], name_cols[x.model2]), axis=1)
pvals_df

,model1,model2,pval,eh,column_latex
0,eakf_model3,2_past_ens,3.561814e-03,1m,Model 3 vs Ens. (2 Months)
1,eakf_model3,all_past_ens,1.166190e-02,1m,Model 3 vs Ens. (All past)
2,2_past_ens,all_past_ens,1.620516e-01,1m,Ens. (2 Months) vs Ens. (All past)
3,eakf_model3,equal_weights_ensemble,5.785863e-11,1m,Model 3 vs Ens. (Eq. Weights)
4,2_past_ens,equal_weights_ensemble,2.938473e-06,1m,Ens. (2 Months) vs Ens. (Eq. Weights)
5,all_past_ens,equal_weights_ensemble,7.883094e-12,1m,Ens. (All past) vs Ens. (Eq. Weights)
6,eakf_model3,2_past_ens,5.159387e-03,2m,Model 3 vs Ens. (2 Months)
7,eakf_model3,all_past_ens,1.803720e-01,2m,Model 3 vs Ens. (All past)
8,2_past_ens,all_past_ens,5.696493e-02,2m,Ens. (2 Months) vs Ens. (All past)
9,eakf_model3,equal_weights_ensemble,9.169867e-18,2m,Model 3 vs Ens. (Eq. Weights)


In [14]:
for idx_row, row in pd.pivot_table(pvals_df, index="column_latex", columns=["eh"], values="pval").iterrows():
    print("{} & {:.2e} & {:.2e} & {:.2e} & {:.2e} & {:.2e} & {:.2e} ".format(idx_row, row["1m"], row["2m"], row["3m"], row["4m"], row["5m"], row["6m"]))

Ens. (2 Months) vs Ens. (All past) & 1.62e-01 & 5.70e-02 & 3.02e-02 & 1.94e-02 & 1.47e-03 & 2.34e-02 
Ens. (2 Months) vs Ens. (Eq. Weights) & 2.94e-06 & 1.38e-08 & 4.33e-11 & 4.82e-15 & 7.78e-18 & 1.98e-18 
Ens. (All past) vs Ens. (Eq. Weights) & 7.88e-12 & 2.85e-17 & 1.81e-21 & 1.05e-23 & 4.14e-25 & 3.05e-24 
Model 3 vs Ens. (2 Months) & 3.56e-03 & 5.16e-03 & 4.52e-02 & 6.91e-03 & 1.00e-03 & 1.48e-01 
Model 3 vs Ens. (All past) & 1.17e-02 & 1.80e-01 & 7.08e-01 & 2.37e-01 & 7.67e-02 & 7.55e-01 
Model 3 vs Ens. (Eq. Weights) & 5.79e-11 & 9.17e-18 & 6.55e-22 & 3.83e-23 & 3.82e-24 & 1.86e-22 


In [15]:
print("{:.2e}".format(5/100))

5.00e-02
